## tf.data API를 사용한 더 나은 성능
##### url: https://www.tensorflow.org/guide/data_performance?hl=en#overview

- 목차
    - 개요
    - 자원
    - 설정
        - 데이터셋
        - 학습 루프
    - 성능 최적화
        - 간단한 접근법
        - 프리페칭
        - 데이터 추출 병렬화
        - 데이터 변환 병렬화
        - 캐싱
        - 매핑 함수 벡터화하기
        - 메모리 사용 흔적 줄이기
    - 모범 사례 요약
## 개요
GPU들과 TPU들은 단일 훈련 스탭을 수행하기 위한 소요 시간을 급격히 줄일 수 있습니다. <br>
최고 성능을 달성하는 것은 현재 스텝이 끝나기 전에 다음 스텝의 데이터를 운반하는 효율적인 입력 파이프라인이 요구됩니다.<br>
`tf.data` API는 유연하고, 효율적인 입력 파이프라인을 생성하는 것을 도와줍니다.<br>
해당 문서는 매우 성능이 좋은 텐서플로우 입력 파이프라인을 생성하기 위해 `tf.data` API를 어떻게 사용하는지 보여줍니다.<br><br>다음으로 넘어가기 전에 `tf.data` API를 어떻게 사용하는지 배울 수 있는 "[Build TensorFlot input pipelines](https://www.tensorflow.org/guide/data)" 가이드를 읽어보세요.

## 자원
- [Build TensorFlot input pipelines](https://www.tensorflow.org/guide/data)
- [`tf.data.Dataset'](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) API

## 설정

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import time

해당 가이드 전반에서, 반복적으로 데이터셋과 성능 측정을 할 예정입니다.<br>
다른 인자에 의해, 동일한 성능의 벤치마크를 만드는 것은 어려울 수 있습니다.

- 현재 CPU 로드율
- 네트워크 트래픽
- 캐시 같은 복잡한 매카니즘 등

동일한 성능의 벤치마크를 제공하기 위해 인공적으로 예제를 만들겠습니다.

### 데이터 셋
`tf.data.Dataset`를 상속 받은 `ArtificialDataset` 클래스를 정의합니다.<br>
해당 데이터셋은 다음과 같은 특징을 갖습니다.

- `num_samples` 만큼의 샘플을 생성합니다. (기본값 3)
- 파일을 열고 첫 번째 항목을 읽기 전에 잠시 대기합니다.
- 파일에서 데이터를 읽어와 각 아이템을 반환할 때마다 잠시 대기합니다.

In [2]:
class ArtificialDataset(tf.data.Dataset):
    def _generator(num_samples):
        # 파일 열기
        time.sleep(0.03)
        
        for sample_idx in range(num_samples):
            # 파일로부터 (line, record)로 이루어진 데이터를 읽습니다.
            time.sleep(0.015)
            
            yield (sample_idx, )
    
    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=tf.dtypes.int64,
            output_shapes=(1,),
            args=(num_samples,)
        )

이 데이터셋은 `tf.data.Datast.range`와 비슷하지만, 각 샘플의 시작과 사이에 고정된 딜레이가 추가되었습니다.

### 학습 루프
전반에 걸쳐 데이터셋을 반복하는 것이 얼마나 오래걸리는지 측정하기 위해 더미 학습 루프를 작성해야합니다.<br>
학습 시간을 시뮬레이팅 하겠습니다.

In [3]:
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # 학습을 수행 하는 단계
            time.sleep(0.01)
    tf.print('수행 시간: ', time.perf_counter() - start_time)

## 성능 최적화하기
어떻게 성능이 최적화될 수 있는지 보여주기 위해서, `ArtificialDataset`의 성능을 개선해야 합니다.

### 간단한 접근 방법
트릭을 사용하지 않고 간단한 파이프라인으로 시작해봅시다. 현재 데이터셋은 다음과 같이 반복됩니다.

In [4]:
benchmark(ArtificialDataset())

수행 시간:  0.2561568249948323


아래와 같이, 수행 시간이 어떻게 소비되었는지 볼 수 있습니다.

<img src=https://www.tensorflow.org/guide/images/data_performance/naive.svg />

학습 단계를 수행하는 것은 다음을 포함합니다. 
- 파일이 열리지 않았다면, 파일을 연다.
- 데이터 엔트리를 파일로부터 불러온다.
- 학습을 위해 데이터를 사용한다.

이와 같이 간단하게 동기화된 구현에서 모델은 파이프라인이 데이터를 파일로부터 불러오는 동안 유휴(idle) 상태에 있게 됩니다.<br>
반대로 모델이 학습하는 동안 입력 파이프라인은 유휴 상태에 있게 됩니다.<br>
이와 같이 학습 단계의 소요 시간은 파일 열기, 데이터 불러오기, 학습 시간의 합으로 이루어집니다.<br><br>
다음 섹션에서는 입력 파이프라인을 구축하고, 성능이 뛰어난 텐서플로우 입력 파이프라인 설계의 모범 사례를 보여줍니다.